In [ ]:
# default_exp recursion

# recursion

> This module contains recursive algorithms for accessing, sorting, and analyzing most python objects. The most powerful functions are `get`, `give`, `sort`, and `maps`. Useful utils include `depth`, `flatten`, and `pipe`.

In [ ]:
#hide
from nbdev import *
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import typing
import numpy as np
from typing import Optional, Tuple, Dict, Callable, Union, Mapping, Sequence, Iterable
from functools import partial
import warnings
from sidis.conversion import cast,convert
import collections

# depth and flattening
> For nested lists and dicts, it's often convenient to know how deep the rabbit hole goes, and extract every element into a single iterable.

In [ ]:
#exports
def depth(obj : Union[dict,list]):
    '''
    Recursively sort the number of layers of a nested
    list or dictionary `x`.
    '''
    if type(obj) is dict and obj:
        return 1 + max(depth(obj[a]) for a in obj)
    if type(obj) is list and obj:
        return 1 + max(depth(a) for a in obj)
    return 0


In [ ]:
show_doc(depth)

<h4 id="depth" class="doc_header"><code>depth</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>depth</code>(**`obj`**:`Union`\[`dict`, `list`\])

Recursively sort the number of layers of a nested
list or dictionary `x`.

In [ ]:
depth([0,[1,[2]]])

3

In [ ]:
depth({0:{0:{0:0}}})

3

In [ ]:
#export
def flatten(obj : Union[dict,list], parent_key='',sep=','):
    '''
    Concatenate the nested input `obj` into an equivalent
    datastructure of depth 1. Uses the `parent_key` and `sep`
    arg to combine nested dictionary keys.
    https://stackoverflow.com/questions/6027558/flatten-nested-dictionaries-compressing-keys
    
    '''

    def flatten_list(obj):
        if obj == []:
            return obj
        if isinstance(obj[0], list):
            return flatten_list(obj[0]) + flatten_list(obj[1:])
        return obj[:1] + flatten_list(obj[1:])
    
    def flatten_dict(obj=obj,parent_key=parent_key,sep=sep):
        items = []
        for k, v in obj.items():
            new_key = parent_key + sep + str(k) if parent_key else str(k)
            if isinstance(v, collections.abc.MutableMapping):
                items.extend(flatten_dict(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)
    
    if type(obj) is dict:
            return flatten_dict(obj)
    elif type(obj) is list:
        return flatten_list(obj)


In [ ]:
show_doc(flatten)

<h4 id="flatten" class="doc_header"><code>flatten</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>flatten</code>(**`obj`**:`Union`\[`dict`, `list`\], **`parent_key`**=*`''`*, **`sep`**=*`','`*)

Concatenate the nested input `obj` into an equivalent
datastructure of depth 1. Uses the `parent_key` and `sep`
arg to combine nested dictionary keys.
https://stackoverflow.com/questions/6027558/flatten-nested-dictionaries-compressing-keys

In [ ]:
flatten([0,[1,[2,[3]]]])

[0, 1, 2, 3]

In [ ]:
i=flatten({'a':0,'b':{'c':1,'d':{'e':2}}})
print(i)

{'a': 0, 'b,c': 1, 'b,d,e': 2}


In [ ]:
#export
def unflatten(d : dict, sep=","):
    '''
    Un-flatten a flattened nested dictionary `d` with
    concatenated key separation `sep`.
    https://gist.github.com/fmder/494aaa2dd6f8c428cede
    '''
    items = dict()
    for k, v in d.items():
        keys = k.split(sep)
        sub_items = items
        for ki in keys[:-1]:
            try:
                sub_items = sub_items[ki]
            except KeyError:
                sub_items[ki] = dict()
                sub_items = sub_items[ki]
            
        sub_items[keys[-1]] = v

    return items

In [ ]:
show_doc(unflatten)

<h4 id="unflatten" class="doc_header"><code>unflatten</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>unflatten</code>(**`d`**:`dict`, **`sep`**=*`','`*)

Un-flatten a flattened nested dictionary `d` with
concatenated key separation `sep`.
https://gist.github.com/fmder/494aaa2dd6f8c428cede

In [ ]:
unflatten(i)

{'a': 0, 'b': {'c': 1, 'd': {'e': 2}}}

## get and give
> These functions access and assign elements, attributes, methods, and function calls of arbitrary python objects and return what sticks.

In [ ]:
#export
def get(obj : Union[object,dict,list,tuple,callable,np.ndarray],
        *args : Union[None,str,int,float,tuple,list,dict],
        retnone : bool = True,
        call : bool = False,
        **kwargs
       ) -> Union[object,dict,list,tuple,callable,int,float,set,np.ndarray]:
    '''
    Recursively accesses `obj` by the given ordered attributes/keys/indexes `args`.
    
    If `call`, the last accessed object will try to call any **kwargs.
    
    If `retnone`, returns None if 
    the object can't be accessed by any of the args; else, returns `obj`. 
    '''
    res=obj
    for attr in args:
        try: #list, arr, tuple w/index=attr, dict w/ key=attr
            res=res[attr]
        except: 
            try: #class object w/ attr
                res=getattr(res,attr)
            except: #method or callable
                try:
                    res=res(attr)
                except:
                    pass
                
    if call:
        try:
            res=res(**kwargs)
        except:
            res=res
    
    if res is not obj:
        return res
    else:
        return (None if retnone else obj)

In [ ]:
show_doc(get)

<h4 id="get" class="doc_header"><code>get</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>get</code>(**`obj`**:`Union`\[`object`, `dict`, `list`, `tuple`, `callable`, `ndarray`\], **\*`args`**:`Union`\[`NoneType`, `str`, `int`, `float`, `tuple`, `list`, `dict`\], **`retnone`**:`bool`=*`True`*, **`call`**:`bool`=*`False`*, **\*\*`kwargs`**)

Recursively accesses `obj` by the given ordered attributes/keys/indexes `args`.

If `call`, the last accessed object will try to call any **kwargs.

If `retnone`, returns None if 
the object can't be accessed by any of the args; else, returns `obj`. 

In [ ]:
a=np.array([[0,1],[2,3]])
get(a,0)

array([0, 1])

In [ ]:
get(a,'doesnt exist')

In [ ]:
get(a,'doesnt exist',retnone=False)

array([[0, 1],
       [2, 3]])

In [ ]:
get(a,'mean')

<function ndarray.mean>

In [ ]:
get(a,'mean',-1)

array([0.5, 2.5])

In [ ]:
get(a,'mean',call=True,axis=-1)

array([0.5, 2.5])

In [ ]:
get(a,'mean',call=True)

1.5

In [ ]:
get(a,'mean','doesnt exist',call=True)

1.5

This function is extremely powerful. Because it tries to access different data structures, and skips any it can't access, `get` can be used in very flexible and exploratory programming styles in order to extract data and perform transformations. 

Next, we will use a similar method to insert arbitrary methods and data into objects with `give`.

In [ ]:
#export
def give(obj : Union[object,dict,list,tuple,np.ndarray],
        *args : Union[str,int,float,tuple,list,dict],
         **kwargs : object
       ) -> Union[object,dict,list,tuple,np.ndarray]:
    '''
    Assign `args` element of `obj` to a value. If no `kwargs`,
    assign the last item of `args`. If `kwargs`, assign those.
    '''

    if kwargs=={}:# no key-value pairs to set, use last ordered arg, assuming len(args)>1:
        if len(args)<2:
            print('Not enough args or kwargs - require access to elements of obj.')

        elif len(args)==2: #index by first, set value of second
            try:
                obj[args[0]]=args[1]
            except: 
                try: 
                    setattr(obj,args[0],args[1])
                except: 
                    print(f"Could not access up to {args[:-1]} and/or apply {args[-1]}.")
                    
        elif len(args)>2: #get up to -3, access by -2, set value with -1
            temp=get(obj,*args[:-2]) #need to access temporary variable
            try: 
                temp[args[-2]]=args[-1]
            except: 
                try: 
                    setattr(temp,args[-2],args[-1])
                except: 
                    print(f"Could not access up to {args[:-1]} and/or apply {args[-1]}.")
                
    else: #dict or obj-like
        if len(args)==0: #set key,value pairs or attrs
            try:
                for k,v in kwargs.items():
                    try:
                        obj[k]=v
                    except:
                        setattr(obj,k,v)
            except:
                print(f"Could not access {k} and/or apply {v}.")

        else:
            try:
                temp=get(obj,*args,retnone=False)
                for k,v in kwargs.items():
                    try:
                        temp[k]=v
                    except:
                        setattr(temp,k,v)
            except:
                print(f"Could not access {k} and/or apply {v}.")



In [ ]:
show_doc(give)

<h4 id="give" class="doc_header"><code>give</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>give</code>(**`obj`**:`Union`\[`object`, `dict`, `list`, `tuple`, `ndarray`\], **\*`args`**:`Union`\[`str`, `int`, `float`, `tuple`, `list`, `dict`\], **\*\*`kwargs`**:`object`)

Assign `args` element of `obj` to a value. If no `kwargs`,
assign the last item of `args`. If `kwargs`, assign those.

In [ ]:
a=[0]
give(a,0,1) #give the 0th element of a the value 1
print(a)

[1]


`give` only works on indexable objects:

In [ ]:
a=0
give(a,0)

Not enough args or kwargs - require access to elements of obj.


In [ ]:
a=np.array([[0,1],[2,3]])
give(a,(1,1),100)
a #arrays must be accessed using tuples

array([[  0,   1],
       [  2, 100]])

In [ ]:
a=[[0,1],[2,3]]
give(a,1,1,100)
a #lists must be accessed using sequences

[[0, 1], [2, 100]]

`give` can be used to replace loops:

In [ ]:
a=[0]
[give(a,0,a[0]+1) for i in range(10)]
print(a)

[10]


Let's use a more complex example. We'll use a networkx graph object to create a simple ring of nodes 0->1->2->0 and apply some attributes.

In [ ]:
import networkx as nx
g=nx.DiGraph()
g.add_edges_from([(0,1),(1,2),(2,0)])

In [ ]:
[get(g.edges,e,['weight']) for e in g.edges] #normally this would produce an error!

[{}, {}, {}]

In [ ]:
[give(g.edges,e,weight=np.random.random(1)) for e in g.edges] #give returns none

[None, None, None]

In [ ]:
for e in g.edges: #newly applied values
    print(e,g.edges[e]['weight'])

(0, 1) [0.15324507]
(1, 2) [0.15174739]
(2, 0) [0.24494159]


In [ ]:
[get(g.nodes,n,'a') for n in g.nodes] #normally would give error!

[{}, {}, {}]

In [ ]:
[give(g.nodes,n,a=np.random.random(1)) for n in g.nodes]
[get(g.nodes,n,'a') for n in g.nodes]

[array([0.87330487]), array([0.23282991]), array([0.25588777])]

These functions can be used in many more ways than those shown here, and encourage exploratory programming. 

## sort

> Now we examine sorting data structures with `sort` using `get`.

In [ ]:
#export
def sort(obj : Iterable,
         *args : Union[None,str,int,float,tuple,list,dict],
         by : Union[None,object,dict,list,tuple,callable,np.ndarray] = None,#lambda o: o,
         key : callable = lambda t: get(t,-1,retnone=False) if get(t,-1,retnone=False) else 0,#lambda t:t[-1] if depth(t)!=0 else t,
         sift : callable = lambda t: True,# if get(t,-1,retnone=False) else False,
         reverse : bool = True
        ) -> Union[None,int,float,list,tuple,str,dict,np.ndarray]:
    '''
    Recursively sorts `obj` `by` `args` using `key`.
    
    Treats `obj` as an iterator and wraps `by` around the elements of `obj`,
    
    before optionally wrapping any remaining `args`, and returning the evaluated 
    
    tuples generated over `obj`. If `by` is None, searches for `args`. 
    
    If both `by` and `args` is None, sorts over `obj` only : default behavior.
    
    `sift` defaults to keeping all elements, while
    
    `key` defaults to treating empty objects as having value 0.
    
    `reverse` sorts ascending by default.
    
    '''
    
    if by is not None:#if you're sorting over a different evaluation than the object elements
    
        sorting=filter(sift,[ (i, get( get( by, i, retnone=False), #evaluate inner function
                        *args,retnone=False) #and any remaining args
                  ) for i in obj]) #over iterable and store tuples then sort
    
    elif args is not (): #if you're sorting over the object and provide args
        
        sorting=filter(sift,[get(get(obj,i,retnone=False),*args,retnone=False) for i in obj])
    
    else: #otherwise you're just sorting the object
        
        sorting=filter(sift,obj)
    
    return sorted(sorting,key=key,reverse=reverse)


In [ ]:
show_doc(sort)

<h4 id="sort" class="doc_header"><code>sort</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>sort</code>(**`obj`**:`Iterable`\[`T_co`\], **\*`args`**:`Union`\[`NoneType`, `str`, `int`, `float`, `tuple`, `list`, `dict`\], **`by`**:`Union`\[`NoneType`, `object`, `dict`, `list`, `tuple`, `callable`, `ndarray`\]=*`None`*, **`key`**:`callable`=*`<lambda>`*, **`sift`**:`callable`=*`<lambda>`*, **`reverse`**:`bool`=*`True`*)

Recursively sorts `obj` `by` `args` using `key`.

Treats `obj` as an iterator and wraps `by` around the elements of `obj`,

before optionally wrapping any remaining `args`, and returning the evaluated 

tuples generated over `obj`. If `by` is None, searches for `args`. 

If both `by` and `args` is None, sorts over `obj` only : default behavior.

`sift` defaults to keeping all elements, while

`key` defaults to treating empty objects as having value 0.

`reverse` sorts ascending by default.

In [ ]:
sort([9,0,1,8,7])

[9, 8, 7, 1, 0]

In [ ]:
sort([9,0,1,8,7],by=lambda t:t%3)

[(8, 2), (1, 1), (7, 1), (9, 0), (0, 0)]

In [ ]:
sort(g.nodes,'a')

[array([0.87330487]), array([0.25588777]), array([0.23282991])]

In [ ]:
sort(g.nodes,'a',by=g.nodes)

[(0, array([0.87330487])), (2, array([0.25588777])), (1, array([0.23282991]))]

In [ ]:
sort(g.edges,'weight')

[array([0.24494159]), array([0.15324507]), array([0.15174739])]

In [ ]:
sort(g.edges,'weight',by=g.edges)

[((2, 0), array([0.24494159])),
 ((0, 1), array([0.15324507])),
 ((1, 2), array([0.15174739]))]

In [ ]:
g.add_edges_from([(0,0)]) #add another edge
sort(g.nodes,by=g.edges,key=lambda t:len(sort(list(t[-1])))) #then sort nodes over edges by in-degree

[(0, OutEdgeDataView([(0, 1), (0, 0)])),
 (1, OutEdgeDataView([(1, 2)])),
 (2, OutEdgeDataView([(2, 0)]))]

In [ ]:
sort(g.nodes,by=g.in_degree) #same as above but relies on 'in_degree' attribute of networkx

[(0, 2), (1, 1), (2, 1)]

In conclusion, `sort` is capable of handling an immense number of possible data structures, and it's best understood by playing around with it and seeing what works! 

A word of caution though: the more complex and custom the python object, the more difficult it is to typecast. Remember to transform your data so that the `key` lambda performs a valid comparison - since it's a lambda function, it's still up to you to make sure the data types are actually comparable in a way that admits a binary operation.

In [ ]:
#export
def pipe(func,otype=None,ftype=None,cast=cast,*args,**kwargs):
    '''
    Pipelines the `func` to act on a later object.
    Returns a partially evaluated function over any `args` and `kwargs`.
    The object is casted to type `otype` before being evaluated.
    The output of the function is casted to `ftype`
    '''
    f=partial(func,*args,**kwargs)
    def line(obj):
        obj=cast(obj,otype)
        return cast(f(obj),ftype)
    return line


In [ ]:
show_doc(pipe)

<h4 id="pipe" class="doc_header"><code>pipe</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>pipe</code>(**`func`**, **`otype`**=*`None`*, **`ftype`**=*`None`*, **`cast`**=*`<sidis.conversion.Caster object at 0x000002C6AECFBA08>`*, **\*`args`**, **\*\*`kwargs`**)

Pipelines the `func` to act on a later object.
Returns a partially evaluated function over any `args` and `kwargs`.
The object is casted to type `otype` before being evaluated.
The output of the function is casted to `ftype`

In [ ]:
pipe(lambda x,y:x+y,otype=int,ftype=int,y=1.9)(1.9) #convert the input to int, and the output to int

2

Pipe is useful for converting function datatypes and passing them as arguments to other functions, as follows:

In [ ]:
#exports
def maps(obj,
             *funcs,
             depth=0,
             zipit=False,
             to=None,
             squeeze=True):
    '''
    Sequentially map `funcs` over the elements of `obj`, "o".
    The first `depth` number of funcs are mapped sequentially f(g(h(...(o)..)))=x.
    The remaining number of funcs are mapped separately (u(x),v(x),...).
    Use partial `funcs` to fill in all args but `obj` if other parameters needed.
    If `keys`, return tuples of the object elements "o" along with map outputs.
    '''
    if not funcs:
        return obj
    else:
        obj=obj if hasattr(obj,'__iter__') and type(obj)!='str' else [obj] #asiter(obj)
        r=[o for o in obj]
        [[give(r,i,get(f,r[i])) for f in funcs[:depth]] for i in range(len(r))]
        [give(r,i,[get(f,r[i]) for f in funcs[depth:]]) for i in range(len(r))]
        if squeeze:
            r=np.ndarray.tolist(np.squeeze(np.array(r,dtype=object)))
        if zipit:
            r=list(zip(obj,r))
        return cast(r,to)

In [ ]:
show_doc(maps)

<h4 id="maps" class="doc_header"><code>maps</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>maps</code>(**`obj`**, **\*`funcs`**, **`depth`**=*`0`*, **`zipit`**=*`False`*, **`to`**=*`None`*, **`squeeze`**=*`True`*)

Sequentially map `funcs` over the elements of `obj`, "o".
The first `depth` number of funcs are mapped sequentially f(g(h(...(o)..)))=x.
The remaining number of funcs are mapped separately (u(x),v(x),...).
Use partial `funcs` to fill in all args but `obj` if other parameters needed.
If `keys`, return tuples of the object elements "o" along with map outputs.

In [ ]:
maps(0,lambda t:t+1,lambda t:t+1,lambda t:t+1,depth=0) #add 1 separately to 0, three times

[1, 1, 1]

In [ ]:
maps(0,lambda t:t+1,lambda t:t+1,lambda t:t+1,depth=1) #add 1 to 0, then add 1 to that separately twice

[2, 2]

In [ ]:
maps(0,lambda t:t+1,lambda t:t+1,lambda t:t+1,depth=-1) #sequantially apply all functions

3

In [ ]:
maps([0,1],lambda t:t+1,lambda t:t+1,lambda t:t+1) #apply functions over elements of iterable

[[1, 1, 1], [2, 2, 2]]

In [ ]:
maps([0,1],lambda t:t+1,lambda t:t+1,lambda t:t+1,depth=-1)

[3, 4]

In [ ]:
maps([0,1],lambda t:t+1,lambda t:t+1,lambda t:t+1,depth=-1,zipit=True) #zip the arguments with their func outputs

[(0, 3), (1, 4)]

In [ ]:
maps(g.nodes,pipe(g.predecessors,None,list),pipe(g.successors,None,list),zipit=True) #use a pipeline

[(0, [[2, 0], [1, 0]]), (1, [[0], [2]]), (2, [[1], [0]])]

In [ ]:
maps(g.nodes,pipe(g.predecessors,None,list),pipe(g.successors,None,list),zipit=True,to=dict) #convert the output

{0: (0, [[2, 0], [1, 0]]), 1: (1, [[0], [2]]), 2: (2, [[1], [0]])}

In [ ]:
#hide
notebook2script()

Converted 00_utils.ipynb.
Converted 01_conversion.ipynb.
Converted 02_recursion.ipynb.
Converted 03_templates.ipynb.
Converted index.ipynb.
